In [1]:
from pathlib import Path
from time import time
import pandas as pd
from videosentimentanalysis.usecases.download_video_use_case import VideoUseCase
from videosentimentanalysis.adapters.logging.custom_logger import CustomLogger
from videosentimentanalysis.adapters.logging.loguru_logger import LoguruLogger
from videosentimentanalysis.adapters.extract_audio.moviepy_extraction import MoviePyAudioExtractor
from videosentimentanalysis.usecases.video_utility_use_case import VideoUtilityUseCase
from videosentimentanalysis.adapters.transcribe_audio.speechrecognition import SpeechRecognition

EXPLAIN HERE THE ARCHITECTURE OF THE PROJECT

To run this project you need to have the following installed:
- ffmpeg
- python3.12
- poetry

## Setup Adapters
### Logging


In [2]:
custom_logger = CustomLogger(Path("./download_log.txt"))
loguru_logger = LoguruLogger(Path("./loguru_logger.log"))

### Video Utilities

In [3]:
moviepy_extractor = MoviePyAudioExtractor(Path("./tmp_audio"))
speechrecognition = SpeechRecognition()

Load all the Youtube URLs to a Pydantic Model called RawURLVideo

In [4]:
video_use_case = VideoUseCase(video_output_directory=Path("tmp_videos"), logger=custom_logger)

In [5]:
raw_url_videos=video_use_case.load_raw_url_video(Path('./video_urls.txt'))
raw_url_videos

[RawVideoUrl(url='https://www.youtube.com/watch?v=G025oxyWv0E\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=G7xFfezsJ1s')]

In [6]:
sequential_time_taken = time()
video_use_case.download_raw_videos_sequentially(raw_url_videos)
sequential_time_taken = time() - sequential_time_taken

2024-06-14 20:24:40.469028+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-14 20:24:42.072330+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/4dd0e83f-d713-4d72-88a1-72f9849f8f94If Educational Videos Were Filmed Like Music Videos.mp4 {} ()
2024-06-14 20:24:42.200973+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-14 20:24:43.291415+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/f771cfbe-5f88-47ab-869f-4dfde5927a87Casually Explained Guide to College and University.mp4 {} ()
2024-06-14 20:24:43.407761+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=T

In [7]:
threading_time_taken = time()
video_use_case.download_raw_videos_threading(raw_url_videos)
threading_time_taken = time() - threading_time_taken

2024-06-14 20:24:48.346422+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' {} ()2024-06-14 20:24:48.345899+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()

2024-06-14 20:24:48.356101+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-14 20:24:48.358376+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-14 20:24:48.405314+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' {} ()
2024-06-14 20:24:50.594430+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/6e5d0532-3c04-4149-ba96-27c5407508b1Fastest Rap EVER.mp4 {} ()
2024-06-14 20:24:50.707455+00:00 INFO: Downlo

In [8]:
process_time_taken = time()
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)
process_time_taken = time() - process_time_taken

2024-06-14 20:24:52.082368+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' {} ()
2024-06-14 20:24:53.268143+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/2ac36c05-2403-40f4-b84d-8fe070e3835bTHE PARTS OF A CELL SONG  Science Music Video.mp4 {} ()
2024-06-14 20:24:52.093739+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' {} ()
2024-06-14 20:24:53.408081+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/e910cd8f-43db-4e37-beea-df40d138966dFastest Rap EVER.mp4 {} ()
2024-06-14 20:24:52.067192+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-14 20:24:53.

## Results of each method of downloading videos and time taken

In [9]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [sequential_time_taken, threading_time_taken, process_time_taken]
})

,Method,Time Taken
0,Sequential,7.887942
1,Threading,3.529059
2,Multiprocessing,2.944262


Blah Blah Blah Monkey stuff

In [10]:
for video in videos:
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=loguru_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition)
    print(video_utlity_use_case.extract_audio())
    print(video_utlity_use_case.transcribe_audio())

2024-06-14 21:24:54.714 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Extracting audio from video If Educational Videos Were Filmed Like Music Videos


MoviePy - Writing audio in tmp_audio/If Educational Videos Were Filmed Like Music Videos.wav


2024-06-14 21:24:55.321 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Returning cached audio for video If Educational Videos Were Filmed Like Music Videos
2024-06-14 21:24:55.322 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:log:14 - Transcribing audio from video If Educational Videos Were Filmed Like Music Videos


MoviePy - Done.
local_storage_path=PosixPath('tmp_audio/If Educational Videos Were Filmed Like Music Videos.wav')


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 46.1MiB/s]
2024-06-14 21:25:10.948 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Extracting audio from video Casually Explained: Guide to College and University


...

MoviePy - Done.
local_storage_path=PosixPath('tmp_audio/THE PARTS OF A CELL SONG | Science Music Video.wav')


2024-06-14 21:25:47.775 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Extracting audio from video THE ROCK CYCLE SONG | Science Music Video


 Here's what he so contains Out of layers of salmon brain It controls what goes in and down Sells that synergy from source of mitochondria Also known as the powerhouse In the ribosomes after cell with proteins in the cis and double as micrarchiculomocary substances Bricking down multi-reels, lies also The nucleus controls the cell kind of like a brain contains a DNA when cells divide to replicate And suddenly a man reign full of pores And nucleolus makes bubbles and minerals from proteins Acreting hormones by the Goldie bodies Each unit of a cell is in organelle The membrane keeps substances out inside Some film inside, or plasma, some bizzostomes Help break things down energy from mitochondria Protein factories, ribosomes, endoplasmic particulum Chance receptances like proteins, degrees controls, please Salmon brain, mitochondria, lysosomes, ribosomes, cytoplasm, nucleus, ER-ingled-day body And then nucleolus, salmon brain, mitochondria, lysosomes, ribosomes, cytoplasm, nucleus, ER-in

2024-06-14 21:25:48.454 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:info:17 - Returning cached audio for video THE ROCK CYCLE SONG | Science Music Video
2024-06-14 21:25:48.455 | INFO     | videosentimentanalysis.adapters.logging.loguru_logger:log:14 - Transcribing audio from video THE ROCK CYCLE SONG | Science Music Video


MoviePy - Done.
local_storage_path=PosixPath('tmp_audio/THE ROCK CYCLE SONG | Science Music Video.wav')
 Okay now, let's start with minerals, solid naturally Yucca, reeks, substances made from elements, singles, or combos of many inorganic means anal, comp from plants, or animals, always a fixed-can-bickle structure, examples of coastal gold. All right, now do it from minerals, let's talk about what a rock is, composed of many different compounds, structures not consistent. There are major types of rocks, in total there are three igneas, metamorphic and sedimentary. First off, it's igneas, one of all canoeuvres, blues, magma, and then it cools down hardens within the earth's class, going to the metamorphic, formed by massive heat or pressure, sometimes they've found by the fault lines where place push against each other. Sedimentary rocks, rocks sediment combining, like a river picking up to the positive pieces, comes to ragged eventually. Inge is comes from a volcano, metamorphic form